In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.types import * 
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import pyspark.sql.functions as F
from pyspark.sql.functions import col,isnan, when, count, coalesce
from pyspark import SparkContext
from pyspark.sql import SQLContext
import json
from functools import reduce
import sys
from cassandra.cluster import Cluster
import os
# from mock.tasks import adiciona_carro
cluster = Cluster(['cassandra'])
session = cluster.connect()

ss = SparkSession.builder.appName("test").getOrCreate()
sql = SQLContext(ss)

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [2]:
session.execute("USE simulacao")

In [3]:
session.execute("select count(*) from simulacao")[0]

/tmp/ipykernel_6580/795349805.py:1: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  session.execute("select count(*) from simulacao")[0]


Row(count=3071)

In [4]:
number = 170

In [5]:
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = "+str(number)+" ALLOW FILTERING;")

In [6]:
result[0]

/tmp/ipykernel_6580/223460175.py:1: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  result[0]


Row(placa='BRA2A34', rodovia='BR-116', tempo_da_simulacao=170.0, pos_x=745.0, pos_y=842.75)

In [7]:
r = list(result)

In [8]:
r

[Row(placa='BRA2A34', rodovia='BR-116', tempo_da_simulacao=170.0, pos_x=745.0, pos_y=842.75),
 Row(placa='URU2Y34', rodovia='BR-116', tempo_da_simulacao=170.0, pos_x=655.0, pos_y=-31.500000000000796),
 Row(placa='URU6M78', rodovia='BR-116', tempo_da_simulacao=170.0, pos_x=475.0, pos_y=106.5),
 Row(placa='PAR5Z67', rodovia='BR-116', tempo_da_simulacao=170.0, pos_x=835.0, pos_y=622.5)]

In [9]:
df = ss.createDataFrame(r)

df.show()

+-------+-------+------------------+-----+-------------------+
|  placa|rodovia|tempo_da_simulacao|pos_x|              pos_y|
+-------+-------+------------------+-----+-------------------+
|BRA2A34| BR-116|             170.0|745.0|             842.75|
|URU2Y34| BR-116|             170.0|655.0|-31.500000000000796|
|URU6M78| BR-116|             170.0|475.0|              106.5|
|PAR5Z67| BR-116|             170.0|835.0|              622.5|
+-------+-------+------------------+-----+-------------------+



In [10]:
number+=1
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = "+str(number)+" ALLOW FILTERING;")
r = list(result)
df2 = ss.createDataFrame(r)
df2.show()

+-------+-------+------------------+-----+------------------+
|  placa|rodovia|tempo_da_simulacao|pos_x|             pos_y|
+-------+-------+------------------+-----+------------------+
|ARG9X01| BR-116|             171.0|475.0|            159.55|
|BRA8M90| BR-116|             171.0|565.0|             145.8|
|COL1R23| BR-116|             171.0|475.0|157.35000000000002|
|BRA0K12| BR-116|             171.0|385.0|            260.05|
|SUR8W90| BR-116|             171.0|475.0| 367.8500000000001|
|GUY7J89| BR-116|             171.0|745.0|            737.75|
|ECU2U34| BR-116|             171.0|655.0|             749.8|
|BRA4Y56| BR-116|             171.0|385.0|  908.250000000001|
|SUR2K34| BR-116|             171.0|745.0|             822.0|
|CHI8Q90| BR-116|             171.0|385.0| 289.5000000000001|
|PAR5Z67| BR-116|             171.0|835.0|             614.5|
+-------+-------+------------------+-----+------------------+



In [11]:
number+=1
result = session.execute("SELECT * FROM simulacao WHERE tempo_da_simulacao = "+str(number)+" ALLOW FILTERING;")
r = list(result)
df25 = ss.createDataFrame(r)
df25.show()

+-------+-------+------------------+-----+------------------+
|  placa|rodovia|tempo_da_simulacao|pos_x|             pos_y|
+-------+-------+------------------+-----+------------------+
|ARG9X01| BR-116|             172.0|475.0|            159.55|
|BOL5P67| BR-116|             172.0|835.0|            795.25|
|COL1R23| BR-116|             172.0|745.0|             748.0|
|CHI0G12| BR-116|             172.0|385.0| 281.5500000000002|
|BRA2A34| BR-116|             172.0|745.0|             866.0|
|URU2Y34| BR-116|             172.0|655.0| 384.4999999999992|
|BRA0K12| BR-116|             172.0|475.0|            223.35|
|SUR8W90| BR-116|             172.0|475.0| 587.8500000000003|
|ECU8G90| BR-116|             172.0|475.0|            146.25|
|URU6M78| BR-116|             172.0|475.0|             168.5|
|BOL3R45| BR-116|             172.0|745.0|             780.0|
|GUY3V45| BR-116|             172.0|475.0|233.45000000000007|
|GUY7J89| BR-116|             172.0|655.0|            710.15|
|ARG3L45

In [12]:
df_empty = ss.createDataFrame([], "placa: string, posicao: int, linha: int, rodovia: string, tempo_da_simulacao: int, velocidade: double, aceleracao: double, posicao_prevista: double, acima_vel: boolean, aplicaMulta: boolean")

In [13]:
import json
params = json.load(open('./mock/parametros.json'))

In [14]:
p = [[key]+list(params[key].values()) for key in params.keys()]

In [15]:
p = ss.createDataFrame(p, ["Rodovia"]+list(params[list(params.keys())[0]].keys()))

p.toPandas()

,Rodovia,sentido1Name,sentido2Name,sentido1Faixas,sentido2Faixas,probabilidadeDeEntradaDeVeiculo,velocidadeMaxima,velocidadeMinima,AceleracaoMaxima,AceleracaoMinima,probabilidadeDeTrocaDeFaixa,probabilidadeColisao,tempoColisao,kilometroInicial,interfaceGrafica,tamanhoCarro
0,BR-040,Brasilia,Rio de Janeiro,3,3,0.9,10,2,0.10,0.01,0.3,0.1,3000,100,True,74
1,BR-116,Fortaleza,Rio Grande do Sul,3,3,0.8,9,2,0.15,0.05,0.5,0.1,3000,100,False,74
2,BR-135,Belo Horizonte,São Luís,3,3,0.9,12,3,0.20,0.05,0.3,0.1,3000,100,False,74
3,BR-393,Cachoeiro de Itapemirim,Barra Mansa,2,2,0.9,8,2,0.10,0.01,0.5,0.1,3000,100,False,74


In [16]:
import queue


In [17]:
fps = 30

def processa_carro(DadosNovos, DadosCarros, colision_tolerance, colision_tolerance_quad, Parametros):
    DadosCarros = DadosCarros.drop('aplicaMulta')

    # Renomeia coluna do dado novo
    DadosNovos = DadosNovos.select(F.col('pos_y').alias('posicao_nova'), F.col('rodovia').alias('rodovia_nova'),
                                   F.col('pos_x').alias('linha_nova'),
                      F.col('tempo_da_simulacao').alias('tempo_da_simulacao_novo'),F.col('placa'))
    data_joined = DadosCarros.join(DadosNovos, on="placa", how='right')
    data_joined = data_joined.withColumn("rodovia", coalesce(col("rodovia_nova"), col("rodovia")))
    data_joined = data_joined.withColumn("linha", coalesce(col("linha_nova"), col("linha")))
    data_joined = data_joined.drop("rodovia_nova", "linha_nova")
    
    data_joined = data_joined.withColumn("diferenca_de_posicao", col("posicao_nova") - col("posicao"))
    
    data_joined = data_joined.withColumn("diferenca_de_horario", (col("tempo_da_simulacao_novo") - col("tempo_da_simulacao"))*fps)

    data_joined = data_joined.withColumnRenamed("velocidade", "velocidade_antiga")

    data_joined = data_joined.withColumn("velocidade", col("diferenca_de_posicao") / col("diferenca_de_horario"))

    data_joined = data_joined.withColumn("diferenca_de_velocidade", col("velocidade") - col("velocidade_antiga"))

    data_joined = data_joined.withColumn("aceleracao", col("diferenca_de_velocidade") / col("diferenca_de_horario"))

    data_joined = data_joined.drop("velocidade_antiga", "posicao", "tempo_da_simulacao",
                     "diferenca_de_posicao", "diferenca_de_horario",
                     "diferenca_de_velocidade")

    data_joined = data_joined.withColumnRenamed("posicao_nova", "posicao")

    data_joined = data_joined.withColumnRenamed("tempo_da_simulacao_novo", "tempo_da_simulacao")

    data_joined = data_joined.withColumn("posicao_prevista", col("posicao")\
                           + col("velocidade")*colision_tolerance\
                           + col("aceleracao")*colision_tolerance_quad)

    Velocidades_Maximas = Parametros.select(F.col('rodovia'), F.col("VelocidadeMaxima"))
    data_joined = data_joined.join(Velocidades_Maximas, on="rodovia", how="left")

    acima_vel_df = data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_antigo'))
    data_joined = data_joined.withColumn("acima_vel", F.when(F.abs(data_joined["velocidade"]) > F.abs(fps/data_joined["VelocidadeMaxima"]), 1).otherwise(0))

    acima_vel_df = acima_vel_df.join(data_joined.select(F.col('placa'), F.col('acima_vel').alias('acima_vel_novo')), on='placa', how="left")
    acima_vel_df = acima_vel_df.withColumn("aplicaMulta", (F.col('acima_vel_antigo')==0) &  (F.col('acima_vel_novo')==1) )

    data_joined = data_joined.join(acima_vel_df.select(F.col('placa'), F.col('aplicaMulta')), on='placa', how='left')

    data_joined = data_joined.drop("VelocidadeMaxima")
    return data_joined
    

In [18]:
df_empty.show()

+-----+-------+-----+-------+------------------+----------+----------+----------------+---------+-----------+
|placa|posicao|linha|rodovia|tempo_da_simulacao|velocidade|aceleracao|posicao_prevista|acima_vel|aplicaMulta|
+-----+-------+-----+-------+------------------+----------+----------+----------------+---------+-----------+
+-----+-------+-----+-------+------------------+----------+----------+----------------+---------+-----------+



In [19]:
df3 = processa_carro(df, df_empty, 1, 0.5, p)

In [20]:
df3 = processa_carro(df2, df3, 1, 0.5, p)

In [21]:
df4 = processa_carro(df25, df3, 1, 0.5, p)

In [22]:
df4

DataFrame[placa: string, rodovia: string, linha: double, aceleracao: double, posicao_prevista: double, acima_vel: int, posicao: double, tempo_da_simulacao: double, velocidade: double, aplicaMulta: boolean]

In [29]:
df_multas = ss.createDataFrame([], "placa: string, tempo_da_simulacao: int, multa_numero : int")

df_multas

DataFrame[placa: string, tempo_da_simulacao: int, multa_numero: int]

In [30]:
df_multas.show()

+-----+------------------+------------+
|placa|tempo_da_simulacao|multa_numero|
+-----+------------------+------------+
+-----+------------------+------------+



In [31]:
from pyspark.sql.window import Window
def aplica_multa(df):
    global df_multas
    df = df.filter(F.col('aplicaMulta') == True)
    df = df.select(F.col('placa'),F.col('tempo_da_simulacao'))
    # Group by 'placa' and get the last value of each column
    df2 = df_multas.groupBy('placa').agg(F.count('placa').alias('multa_numero'))
    df2 = df2.withColumn('multa_numero',F.col('multa_numero')+1)    
    df2 = df2.join(df,['placa'],how='right')
    df2 = df2.withColumn('multa_numero', F.when(F.isnull('multa_numero'), 1).otherwise('multa_numero'))
    df_multas = df2.union(df_multas)
    df_multas.show()

In [ ]:
aplica_multa(df4)
